In [ ]:
from telethon.sync import TelegramClient
import os
import logging
from dotenv import load_dotenv

# Setup logging
logging.basicConfig(level=logging.INFO, filename='telegram_scraping.log', format='%(asctime)s - %(message)s')

# Load API credentials
API_ID = os.getenv('API_ID')
API_HASH = os.getenv('API_HASH')

# List of target channels
channels = [
    'https://t.me/DoctorsET',
    'https://t.me/lobelia4cosmetics',
    'https://t.me/yetenaweg',
    'https://t.me/EAHCI',
]

# Initialize Telegram client
client = TelegramClient('scraper_session', API_ID, API_HASH)

import asyncio

async def scrape_channel(channel_url, retries=5):
    """Scrapes messages and media from a Telegram channel."""
    async with client:
        for attempt in range(retries):
            try:
                channel = await client.get_entity(channel_url)
                messages = await client.get_messages(channel, limit=100)  # Adjust the limit as needed

                for message in messages:
                    logging.info(f"Message ID: {message.id}, Text: {message.text[:50]}")
                    if message.media:
                        # Save media
                        file_path = await message.download_media(file=f'./downloads/{channel.title}/')
                        logging.info(f"Media saved at {file_path}")
                break  # Exit the retry loop if successful
            except Exception as e:
                logging.error(f"Error scraping {channel_url} on attempt {attempt + 1}: {e}")
                if attempt < retries - 1:
                    await asyncio.sleep(2 ** attempt)  # Exponential backoff
                else:
                    logging.error(f"Failed to scrape {channel_url} after {retries} attempts")

async def main():
    for channel in channels:
        logging.info(f"Scraping channel: {channel}")
        await scrape_channel(channel)

if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: This event loop is already running